In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome()  

url = 'https://www.ibba.org/state/kansas/'

driver.get(url)

driver.implicitly_wait(10) 

html_content = driver.page_source

soup = BeautifulSoup(html_content, 'html.parser')

profile = {
    'name': [],
    'link': []
}

brokers_boxes = soup.find_all('div', class_='brokers__box')

for box in brokers_boxes:
    name_tag = box.find('h4', class_='brokers__item--topTitle')
    name = name_tag.get_text(strip=True) if name_tag else 'N/A'

    link_tag = box.find('a', class_='brokers__item--link')
    link_href = link_tag['href'] if link_tag else 'N/A'

    profile['name'].append(name)
    profile['link'].append(link_href)

driver.quit()

print(profile)


{'name': ['John Karg'], 'link': ['https://www.ibba.org/broker-profile/district-of-columbia/washington/john-karg/']}


In [2]:
email_list = []

for each_link in profile['link']:
    # Initialize the WebDriver (ensure you have the correct path or it's in PATH)
    driver = webdriver.Chrome()

    try:
        url = each_link
        driver.get(url)

        driver.implicitly_wait(10)  # Wait for the page to load

        html_content = driver.page_source

        # Use BeautifulSoup to parse the page content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the desired element
        location = soup.find_all('div', class_='brokers__profile--leftPhone')
        if location and len(location) > 1:
            link_tag = location[1].find('a')
            if link_tag:
                # Extract the text content from the <a> tag
                email_text = link_tag.get_text(strip=True)
                
                email_list.append(email_text)
            else:
                print("Link tag not found.")
        else:
            link_tag = location[0].find('a')
            if link_tag:
                email_text = link_tag.get_text(strip=True)
                
                email_list.append(email_text)
    finally:
        driver.quit()


In [3]:
profile['emails'] = email_list 
import pandas as pd
df = pd.DataFrame(profile)
df.head()

,name,link,emails
0,John Karg,https://www.ibba.org/broker-profile/district-o...,jikargjr@gmail.com


In [4]:
df.to_excel('kansas_email.xlsx')